In [147]:
require "hdf5"
require "optim"
require "nn"

In [148]:
f = hdf5.open("data.hdf5", "r")

X_train = f:read("X_train"):all()
Y_train = f:read("Y_train"):all()
X_valid = f:read("X_valid"):all()
Y_valid = f:read("Y_valid"):all()
X_test = f:read("X_test"):all()
nwords = f:read("nwords"):all()[1]
nclasses = f:read("nclasses"):all()[1]


--sentences
X_valid_sen = f:read("X_valid_sen"):all()
X_test_sen = f:read("X_test_sen"):all()

--MEMM
X_train_MEMM = f:read("X_train_MEMM"):all()
X_valid_MEMM = f:read("X_valid_MEMM"):all()
X_valid_sen_MEMM = f:read("X_valid_sen_MEMM"):all()
nfeaturesMEMM = f:read("nfeaturesMEMM"):all()[1]

C = nclasses

In [154]:
function process(inputs, numwords)
    processed = inputs:clone()
    for i = 1, inputs:size(1) do
        processed[i][2] = inputs[i][2] + numwords
    end
    return processed
end

function feval(w_new)
    bsize= batch
    local idx = torch.randperm(X_:size(1)):sub(1,bsize)
    local x = torch.Tensor(bsize, X_:size(2))
    local y_ = torch.Tensor(bsize, 1)   
    for i=1,bsize do
        x[i] = X_[idx[i]]
        
        y_[i] = y[idx[i]]
    end   
    y_ = y_:squeeze()
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()
    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))   
    return loss_w, dl_dw
end

-- Calculates log scores of for a MEMM
-- corresponds to the vector y in the lecture notes
-- observations: a sequence of observations, represented as integers
-- i: timestep for the computed score
function score_memm(input, i)
    local logscore = torch.log(nn.SoftMax():forward(h:forward(input:sub(i,i))))
    return logscore:t()
end

-- Viterbi algorithm for MEMM (slight modification).
-- observations: a sequence of observations, represented as integers
-- logscore: the edge scoring function over classes and observations in a history-based model
function viterbi(observations, logscore)
    local initial = torch.zeros(nclasses,1) + .00001
    initial[8] = 1.0
    initial = initial / torch.sum(initial)
    local n = observations:size(1)
    local max_table = torch.Tensor(n, C)
    local backpointer_table = torch.Tensor(n, C)
    -- first timestep
    -- the initial most likely paths are the initial state distribution
    -- NOTE: another unnecessary Tensor allocation here
    local maxes, backpointers = (initial + predict(observations, 1, logscore)):max(2)--torch.log(initial):max(2)--(init+ emi[observations[1]]):max(2)
    max_table[1] = maxes
    -- remaining timesteps ("forwarding" the maxes)
    for i=2,n do
        -- precompute edge scores
        pred  = (predict(observations, i, logscore)):expand(C,C)
        scores = pred + maxes:view(1,C):expand(C,C)--pred + maxes:view(1, C):expand(C, C)
        -- compute new maxes (NOTE: another unnecessary Tensor allocation here)
        maxes, backpointers = scores:max(2)
        -- record
        max_table[i] = maxes
        backpointer_table[i] = backpointers
        end
    -- follow backpointers to recover max path
    local classes = torch.Tensor(n)
    maxes, classes[n] = maxes:max(1)
    for i=n,2,-1 do
        classes[i-1] = backpointer_table[{i, classes[i]}]
    end   
    return classes:sub(1,-1)
end

-- Returns a table of predicted tags for each sequence
-- X : tensor of sequences by word features
function predict_tags(X)
    local predictions = torch.zeros(X:size(1), X:size(2))
    for i=1,X:size(1) do
        local sen = (X[i]:sub(1,torch.nonzero(X[{{i,i},{},{1,1}}]):size(1))):squeeze()
        --print(sen)

        local p = viterbi(sen, score_memm)
        predictions[{{i,i},{1,p:size(1)}}] = p
    end
    return predictions
end

-- Returns counts of retrived relevance, unretrived relevance, and retrived irrelevance for supervised data
-- p_tags : tensor of predicted tags for each sequence
-- y_tags : labeled tags for each sequence
function predict_fscore(p_tags, y_tags)
    local rel_retrived = {[1]=0,[2]=0,[3]=0,[4]=0,[5]=0}--,[6]=0,[7]=0}
    local rel_notretrived = {[1]=0,[2]=0,[3]=0,[4]=0,[5]=0}--,[6]=0,[7]=0}
    local irrel_retrived = {[1]=0,[2]=0,[3]=0,[4]=0,[5]=0}--,[6]=0,[7]=0}
    for i=1,p_tags:size(1) do
        for j=1,p_tags:size(2) do
            for tag=1,5 do        
                if y_tags[i][j] == tag then
                    rel_notretrived[tag] = rel_notretrived[tag] + 1
                    if p_tags[i][j] == tag then
                        rel_retrived[tag] = rel_retrived[tag] + 1
                    else
                        irrel_retrived[tag] = irrel_retrived[tag] + 1
                    end
                end
            end
        end
    end
    local recall = 0
    local precis = 0
    for i=1,5 do
        local r = (rel_retrived[i]/(rel_retrived[i]+rel_notretrived[i]))
        local p = (rel_retrived[i]/(rel_retrived[i]+irrel_retrived[i]))
        print(i, p, r, 2*p*r/(p+r))
        recall = recall + r
        precis = precis + p
    end
    return recall/5,precis/5
end

-- Returns validation tags in format similar to p_tags
-- valid : correct tags in sequence
function format_valid(valid)
    local y_tags = torch.zeros(X_valid_sen:size(1), X_valid_sen:size(2))
    local sen_idx = 0
    local col_idx = 0
    for i=1,valid:size(1) do
        if valid[i] == 8 then
            sen_idx = sen_idx + 1
            col_idx = 1
        end
        y_tags[sen_idx][col_idx] = valid[i]
        col_idx = col_idx + 1
    end
    return y_tags
end

In [200]:
predicc = viterbi(X_valid, score_memm)

In [175]:
X_test_sen:size()

 1646
  110
    1
[torch.LongStorage of size 3]



In [196]:
function format_test(test)
    local y_tags = torch.zeros(X_test_sen:size(1), X_test_sen:size(2))
    local sen_idx = 0
    local col_idx = 0
    for i=1,test:size(1) do
        if test[i] == 8 then
            sen_idx = sen_idx + 1
            col_idx = 1
        end
        y_tags[sen_idx][col_idx] = test[i]
        col_idx = col_idx + 1
    end
    return y_tags
end

In [197]:
to_kaggle=format_test(predicc)

In [203]:
Y_valid:sub(1,20)

 8
 1
 1
 4
 1
 1
 1
 1
 1
 1
 1
 1
 8
 3
 1
 8
 5
 5
 1
 2
[torch.LongTensor of size 20]



In [201]:
predicc:sub(1,20)

 8
 1
 1
 4
 2
 2
 1
 2
 1
 2
 2
 1
 8
 3
 1
 8
 3
 5
 4
 2
[torch.DoubleTensor of size 20]



In [156]:
y_tags = format_valid(Y_valid)
r, p = predict_fscore(predictions, y_tags)
print(r, p)
print(2*r*p/(r+p))

1	0.96391705903658	0.49081352728279	0.65043457008875	
2	0.43394833948339	0.30262480699949	0.35657974530018	
3	0.44389275074479	0.30742778541953	0.36326696464852	
4	0.39702760084926	0.28419452887538	0.33126660761736	
5	0.25244618395303	0.2015625	0.2241529105126	
0.31732462971544	0.49824638681341	
0.38771816800341	


In [155]:
a =X_valid_sen_MEMM
--prediction = viterbi(a[1]:sub(1,12), h)
predictions = predict_tags(a)

In [150]:
for i = 1,X_valid_sen_MEMM:size(1) do
    X_valid_sen_MEMM[i] = process(X_valid_sen_MEMM[i], nwords)
end
    
X_ = process(X_train_MEMM, nwords)
y = Y_train

In [151]:
addB = nn.Add(nclasses)
lookup = nn.LookupTable(nwords+nclasses,nclasses)
sum = nn.Sum(2)
softmax = nn.LogSoftMax()
h = nn.Sequential()
h:add(lookup)
h:add(sum)
h:add(addB)
mse = nn.CrossEntropyCriterion()

w, dl_dw = h:getParameters()

In [194]:
batch = 1000

-- cycle on data
for i = 1,50000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,w)

    if i % 100 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

loss for iteration 100 is 0.3197730410171	


loss for iteration 200 is 0.32427961382553	


loss for iteration 300 is 0.33960974628709	


loss for iteration 400 is 0.31646359976712	


loss for iteration 500 is 0.34738714773226	


...

loss for iteration 49600 is 0.11861610649552	


loss for iteration 49700 is 0.084924652964964	


loss for iteration 49800 is 0.10621075707921	


loss for iteration 49900 is 0.084957801565486	


loss for iteration 50000 is 0.10754964070079	


In [192]:
-- Writes to file in Kaggle prediction format
-- p : prediction tensor
-- fname : output filename
function kaggle_format(p, fname)
    local f = io.open("predictions/" .. fname, "w")
    f:write("ID,Labels\n")
    for i=1,p:size(1) do
        local s = ""
        local t = {[1]="O",[2]="PER",[3]="LOC",[4]="ORG",[5]="MISC",[6]="MISC",[7]="LOC",[8]="O",[9]="O"}
        local prev_tag = 0
        local prev_idx = 0
        for j=2,p:size(2) do
            if p[i][j] == 0 then
                break
            end
            if p[i][j] ~= 1 then
                if prev_tag == 0 then
                    s = s .. t[p[i][j]] .. "-" .. tostring(j-1)
                elseif p[i][j] == prev_tag and prev_idx == j-1 then
                    s = s .. "-" .. tostring(j-1) 
                else
                    s = s .. " " .. t[p[i][j]] .. "-" .. tostring(j-1)
                end
                prev_tag = p[i][j]
                prev_idx = j
            end
        end
        f:write(i .. "," .. s .. "\n")
    end
    f:close()
end

In [198]:
kaggle_format(to_kaggle, "pred_" .. tostring(os.time()) .. ".csv")

In [82]:
function predict_tags(X)
    local predictions = torch.zeros(X:size(1), X:size(2))
    for i=1,X:size(1) do
        local sen = (X[i]:sub(1,torch.nonzero(X[{{i,i},{},{1,1}}]):size(1))):squeeze()
        local p = viterbi(sen, score_memm)
        predictions[{{i,i},{1,p:size(1)}}] = p
    end
    return predictions
end

 1646
  110
    1
[torch.LongStorage of size 3]

